# Data Loading and preprocessing

In [230]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config

s3 = boto3.client('s3')
bucket_name = "cancer-knowledge-bucket"

response = s3.list_objects_v2(Bucket=bucket_name, Delimiter='/')

if 'CommonPrefixes' in response:
    for prefix in response['CommonPrefixes']:
        print(prefix['Prefix'])

if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No objects found in the bucket.")

Carmen-ClinvarxMutation/
tcga-kg/
TCGA-CDR-SupplementalTableS1.xlsx
TCGASubtype.20170308.tsv
coad_metadata.csv
coad_mut_data.csv
coad_mutation_rates.csv
coad_subtype_data.csv
gene_0.02thr_HR_pval.csv
human_canonical_pways.csv
tcga_maf_filter_binary.Rds


In [118]:
file_key = "gene_0.02thr_HR_pval.csv"
local_file_name = "gene_0.02thr_HR_pval.csv"
s3.download_file(bucket_name, file_key, local_file_name)
print(f"File {local_file_name} downloaded successfully.")

File gene_0.02thr_HR_pval.csv downloaded successfully.


In [141]:
import pyreadr
import numpy as np
import pandas as pd

path_rds = "tcga_maf_filter_binary.Rds"
result = pyreadr.read_r(path_rds)   
for name, df_maf in result.items():
    print(name, type(df_maf))
print("MAF dataframe shape = ", df_maf.shape)


None <class 'pandas.core.frame.DataFrame'>
MAF dataframe shape =  (10295, 21333)


In [142]:
path_cdr = "TCGA-CDR-SupplementalTableS1.xlsx"

xls = pd.ExcelFile(path_cdr)           
print(xls.sheet_names)

df_subtype_cdr = pd.read_excel(path_cdr, sheet_name="TCGA-CDR")  
display("Shape of subtype df = ", df_subtype_cdr.shape)

path_subtype = "TCGASubtype.20170308.tsv"
df_subtype = pd.read_csv(path_subtype, sep='\t')
print("Shape of subtype df = ", df_subtype.shape)

['TCGA-CDR', 'TCGA-CDR_Notes', 'ExtraEndpoints', 'ExtraEndpoints_Notes', 'Table4_PHAssumptionTests', 'Table5_PHAssumptionTests', 'TSS_Info', 'Fig2EFG_AdditionalInfo']


'Shape of subtype df = '

(11160, 34)

Shape of subtype df =  (7734, 10)


In [246]:
path_pathways = "human_canonical_pways.csv"
df_pathways = pd.read_csv(path_pathways)
print("Shape of subtype df = ", df_pathways.shape)


Shape of subtype df =  (3878, 2094)


In [242]:
path_hr = "gene_0.02thr_HR_pval.csv"
df_hr = pd.read_csv(path_hr)
print("Shape of hr df = ", df_hr.shape)
display(df_hr.head())

Shape of hr df =  (6344, 4)


,Unnamed: 0,gene,exp_hr_coef,hr_pval
0,1,TACC2,0.782922,0.831475
1,2,NAALAD2,0.365278,0.381420
2,3,BTBD11,0.177414,0.052284
3,4,SLC2A14,3.513196,0.283132
4,5,SLC2A3,0.023846,0.010734


In [143]:
df_metadata = pd.read_csv("coad_metadata.csv")
df_mutation = pd.read_csv("coad_mut_data.csv")
df_subtype = pd.read_csv("coad_subtype_data.csv")

In [144]:
print(df_metadata.shape, df_mutation.shape, df_subtype.shape)

(404, 34) (404, 21334) (341, 10)


In [163]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

### Some more preprocessing

In [194]:
def normalize_mutation(df_mutation, barcode_col="Tumor_Sample_Barcode"):
    dm = df_mutation.copy()
    dm = dm.rename(columns={barcode_col: "sample_id"})
    dm["patient_barcode"] = dm["sample_id"].astype(str).str[:12]
    return dm

def normalize_metadata(df_metadata):
    md = df_metadata.copy()
    md = md.rename(columns={"bcr_patient_barcode":"patient_barcode", "type":"cancer_type"})
    display(md.head())
    return md

def normalize_subtype(df_subtype, id_col="pan.samplesID"):
    ds = df_subtype.copy()
    if id_col not in ds.columns:
        raise ValueError(f"df_subtype must have '{id_col}'")
    ds = ds.rename(columns={id_col: "patient_barcode"})
    if "cancer.type" in ds.columns and "tumor_type" not in ds.columns:
        ds = ds.rename(columns={"cancer.type": "tumor_type"})
    return ds

def gene_columns(dm):
    non_gene = {"sample_id","patient_barcode"}
    valid = re.compile(r"^[A-Za-z0-9._-]+$")
    return [c for c in dm.columns if c not in non_gene and valid.match(c)]

def pick_subtype_column(ds, preferred_subtype_cols=None):
    if preferred_subtype_cols is None:
        preferred_subtype_cols = [
            "Subtype_Selected",
        ]
    for c in preferred_subtype_cols:
        if c in ds.columns:
            return c
    return None

# Writing Vertices

In [251]:
def write_vertices_sample(dm, md, out_dir="neptune_csv"):
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)

    sex_map = (
        md[["patient_barcode","gender"]]
        .copy()
        .assign(gender=lambda d: d["gender"].astype(str).str.upper().str.strip())
        .replace({"gender": {"F": "FEMALE", "M": "MALE"}})
    )
    sex_map["sex_norm"] = sex_map["gender"].where(sex_map["gender"].isin(["FEMALE","MALE"]), None)
    sex_map = sex_map[["patient_barcode","sex_norm"]].drop_duplicates()

    v = (
        dm[["patient_barcode"]]
        .drop_duplicates()
        .merge(sex_map, on="patient_barcode", how="left")
    )

    out_df = pd.DataFrame({
        "~id": "sample:" + v["patient_barcode"].astype(str),
        "~label": "Sample",
        "patient_barcode:String": v["patient_barcode"].astype(str),
        "sex:String": v["sex_norm"].fillna(""),   # empty string if unknown
    }).drop_duplicates("~id")

    out_df.to_csv(out / "vertices_sample.csv", index=False)

def write_vertices_age(md, out_dir = "neptune_csv", create_both = True):
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)
    
    
    ages = pd.to_numeric(md.get("age_at_initial_pathologic_diagnosis"), errors="coerce")
    present = set()
    if (ages <= 50).any(): present.add("<=50")
    if (ages  > 50).any(): present.add(">50")

    if create_both:
        present |= {"<=50", ">50"}

    if not present:
        pd.DataFrame(columns=["~id","~label","age_bucket:String"]).to_csv(out/"vertices_age.csv", index=False)
        return

    buckets = pd.Series(sorted(present, key=lambda x: (x != "<=50", x)))
    v = pd.DataFrame({
        "~id": "age:" + buckets,
        "~label": "Age",
        "age_bucket:String": buckets
    })
    v.to_csv(out/"vertices_age.csv", index=False)

def write_vertices_gene(dm, out_dir = "neptune_csv"):
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)
    genes = pd.DataFrame({"symbol": sorted(set(gene_columns(dm)))})
    v = pd.DataFrame({
        "~id": "gene:" + genes["symbol"].astype(str),
        "~label": "Gene",
        "symbol:String": genes["symbol"].astype(str),
    })
    v.to_csv(out/"vertices_gene.csv", index=False)

def write_vertices_gene_with_hr(dm, df_hr, out_dir="neptune_csv"):
    def _sanitize_hr(series):
        x = pd.to_numeric(series, errors="coerce")
        mask = (x >= 0.0) & (x <= 1.0)
        y = x.where(mask)              # out-of-range -> NaN
        y = y.map(lambda v: f"{v:.6g}" if pd.notna(v) else "N/A")  # keep concise; NaN -> "N/A"
        return y

    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)

    genes = pd.DataFrame({"symbol": sorted(set(gene_columns(dm)))})

    hr = df_hr.copy()
    if "gene" not in hr.columns:
        raise ValueError("df_hr must have a 'gene' column")
    cols = ["gene"]
    if "exp_hr_coef" in hr.columns: cols.append("exp_hr_coef")
    if "hr_pval" in hr.columns: cols.append("hr_pval")
    hr = hr[cols].drop_duplicates().rename(columns={"gene": "symbol", "exp_hr_coef": "exp_hr_coeff"})

    # Merge and sanitize to [0,1]
    merged = genes.merge(hr, on="symbol", how="left")
    merged["exp_hr_coeff"] = _sanitize_hr(merged.get("exp_hr_coeff"))
    merged["hr_pval"]      = _sanitize_hr(merged.get("hr_pval"))

    v = pd.DataFrame({
        "~id": "gene:" + merged["symbol"].astype(str),
        "~label": "Gene",
        "symbol:String": merged["symbol"].astype(str),
        "exp_hr_coeff:String": merged["exp_hr_coeff"],
        "hr_pval:String": merged["hr_pval"],
    })
    v.to_csv(out / "vertices_gene.csv", index=False)

def write_vertices_subtype(ds, out_dir = "neptune_csv"):
    out = Path(out_dir) 
    out.mkdir(parents=True, exist_ok=True)

    if "Subtype_Selected" not in ds.columns:
        raise ValueError("Subtype_Selected column not found in subtype dataframe")
    vals = (
        ds["Subtype_Selected"]
        .dropna()
        .astype(str).str.strip()
        .replace({"": np.nan})
        .dropna()
        .drop_duplicates()
    )

    if vals.empty:
        pd.DataFrame(columns=["~id","~label"]).to_csv(out/"vertices_subtype.csv", index=False)
        return

    v = pd.DataFrame({
        "~id": "subtype:" + vals,
        "~label": "Subtype"
    })
    v.to_csv(out/"vertices_subtype.csv", index=False)

def write_vertices_pathway(df_pathways, out_dir="neptune_csv"):
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)
    pathway_cols = [c for c in df_pathways.columns if c != "gene"]
    v = pd.DataFrame({
        "~id": ["pathway:" + p for p in pathway_cols],
        "~label": "Pathway",
        "name:String": pathway_cols,
    })
    v.to_csv(out / "vertices_pathway.csv", index=False)

# Writing Edges

In [252]:
def write_edges_has_age(dm, md, out_dir="neptune_csv"):
    out = Path(out_dir); out.mkdir(parents=True, exist_ok=True)

    tmp = dm[["patient_barcode"]].drop_duplicates().merge(
        md[["patient_barcode","age_at_initial_pathologic_diagnosis"]],
        on="patient_barcode", how="left"
    )
    age = pd.to_numeric(tmp["age_at_initial_pathologic_diagnosis"], errors="coerce")
    bucket = np.where(age > 50, ">50", "<=50")
    tmp["age_bucket"] = pd.Series(bucket)
    tmp = tmp.dropna(subset=["age_at_initial_pathologic_diagnosis"])  

    
    
    e = pd.DataFrame({
        "~id": "e:sample:" + tmp["patient_barcode"].astype(str) + ":" + tmp["patient_barcode"].astype(str),
        "~from": "sample:" + tmp["patient_barcode"].astype(str),
        "~to": "age:" + tmp["age_bucket"].astype(str),
        "~label": "IS_OF_AGE"
    })
    e.to_csv(out/"edges_is_of_age.csv", index=False)


def write_edges_has_gene_streaming(dm, out_dir="neptune_csv", batch_genes=100, split_every=2_000_000):
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)
    genes = gene_columns(dm)
    part, written, fh = 0, 0, None

    def _open():
        nonlocal part, written, fh
        if fh is not None: fh.close()
        part += 1; written = 0
        p = out / f"edges_has_gene_part{part:03d}.csv"
        fh = open(p, "w", buffering=1<<20)
        fh.write("~id,~from,~to,~label,value:Double\n")
        return fh
    fh = _open()

    def present(s):
        if s.dtype.kind in "biufc": return s.fillna(0) != 0
        sv = s.astype(str).str.strip()
        return (~s.isna()) & (~sv.isin(["","0","NA","NaN","nan"]))

    base = dm[["patient_barcode"]]
    for i in range(0, len(genes), batch_genes):
        cols = genes[i:i+batch_genes]
        block = pd.concat([base, dm[cols]], axis=1)
        for g in cols:
            s = block[["patient_barcode", g]].dropna()
            m = present(s[g])
            if not m.any(): continue
            ss = s.loc[m]
            vals = pd.to_numeric(ss[g], errors="coerce")
            for sid, val in zip(ss["patient_barcode"].astype(str), vals):
                if written >= split_every:
                    fh = _open()
                line = f"e:sample:{sid}:{g},sample:{sid},gene:{g},HAS_GENE,"
                fh.write(line + ("" if pd.isna(val) else f"{float(val)}") + "\n")
                written += 1
    fh.close()

def write_edges_has_subtype(dm, ds, out_dir="neptune_csv"):
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)

    subtype_col = "Subtype_Selected"
    pts_with_samples = dm["patient_barcode"].drop_duplicates()
    ds_f = ds[ds["patient_barcode"].isin(pts_with_samples)].copy()

    submap = (
        ds_f[["patient_barcode", subtype_col]]
        .dropna()
        .astype({subtype_col: str})
        .assign(**{subtype_col: lambda d: d[subtype_col].str.strip()})
    )
    submap = submap[submap[subtype_col] != ""].drop_duplicates()

    if submap.empty:
        pd.DataFrame(columns=["~id","~from","~to","~label"]).to_csv(out/"edges_has_subtype.csv", index=False)
        return

    samp = dm[["patient_barcode"]].drop_duplicates()
    j = samp.merge(submap, on="patient_barcode", how="inner")

    e = pd.DataFrame({
        "~id": "e:sample:" + j["patient_barcode"].astype(str) + ":"+j[subtype_col].astype(str),
        "~from": "sample:" + j["patient_barcode"].astype(str),
        "~to": "subtype:" + j[subtype_col].astype(str),
        "~label": "HAS_SUBTYPE"
    })
    e.to_csv(out/"edges_has_subtype.csv", index=False)


def write_edges_gene_in_pathway(df_pathways, out_dir="neptune_csv"):
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)
    pathway_cols = [c for c in df_pathways.columns if c != "gene"]
    edge_path = out / "edges_in_pathway.csv"
    with open(edge_path, "w") as fh:
        fh.write("~id,~from,~to,~label\n")
        for _, row in df_pathways.iterrows():
            g = str(row["gene"])
            for p in pathway_cols:
                val = row[p]
                keep = False
                try:
                    keep = float(val) != 0
                except Exception:
                    keep = str(val).strip() not in ("", "0", "NA", "NaN", "nan")
                if keep:
                    fh.write(f"e:gene:{g}:{p},gene:{g},pathway:{p},IN_PATHWAY\n")


In [254]:
dm = normalize_mutation(df_mutation)
md = normalize_metadata(df_metadata)
ds = normalize_subtype(df_subtype)

print("Writing vertices ...")
write_vertices_sample(dm, md, out_dir="neptune_csv")
write_vertices_age(md, out_dir="neptune_csv", create_both=True)
write_vertices_gene_with_hr(dm, df_hr, out_dir="neptune_csv")
write_vertices_subtype(ds, out_dir="neptune_csv")
print("vertices done")

print("Writing edges ...")
write_edges_has_age(dm, md, out_dir="neptune_csv")
write_edges_has_gene_streaming(dm, out_dir="neptune_csv", batch_genes=100, split_every=2_000_000)
write_edges_has_subtype(dm, ds, out_dir="neptune_csv")
write_edges_gene_in_pathway(df_pathways, out_dir="neptune_csv")
print("edges done")
print("Neptune CSVs ready in ./neptune_csv")

,...1,patient_barcode,cancer_type,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,...,residual_tumor,OS,OS.time,DSS,DSS.time,DFI,DFI.time,PFI,PFI.time,Redaction
0,1954,TCGA-3L-AA1B,COAD,61,FEMALE,BLACK OR AFRICAN AMERICAN,Stage I,[Not Applicable],Colon Adenocarcinoma,[Not Available],...,NaN,0,475.0,0.0,475.0,0.0,475.0,0,475.0,NaN
1,1955,TCGA-4N-A93T,COAD,67,MALE,BLACK OR AFRICAN AMERICAN,Stage IIIB,[Not Applicable],Colon Adenocarcinoma,[Not Available],...,NaN,0,146.0,0.0,146.0,NaN,NaN,0,146.0,NaN
2,1956,TCGA-4T-AA8H,COAD,42,FEMALE,BLACK OR AFRICAN AMERICAN,Stage IIA,[Not Applicable],Colon Mucinous Adenocarcinoma,[Not Available],...,NaN,0,385.0,0.0,385.0,0.0,385.0,0,385.0,NaN
3,1957,TCGA-5M-AAT4,COAD,74,MALE,BLACK OR AFRICAN AMERICAN,Stage IV,[Not Applicable],Colon Adenocarcinoma,[Not Available],...,NaN,1,49.0,1.0,49.0,NaN,NaN,1,49.0,NaN
4,1958,TCGA-5M-AAT6,COAD,40,FEMALE,BLACK OR AFRICAN AMERICAN,Stage IV,[Not Applicable],Colon Adenocarcinoma,[Not Available],...,NaN,1,290.0,1.0,290.0,NaN,NaN,1,219.0,NaN


Writing vertices ...
vertices done
Writing edges ...
edges done
Neptune CSVs ready in ./neptune_csv


# Uploading all local files to Amazon S3

In [257]:
! aws s3 cp neptune_csv/ s3://cancer-knowledge-bucket/tcga-kg/ --recursive

upload: neptune_csv/.ipynb_checkpoints/edges_has_subtype-checkpoint.csv to s3://cancer-knowledge-bucket/tcga-kg/.ipynb_checkpoints/edges_has_subtype-checkpoint.csv
upload: neptune_csv/.ipynb_checkpoints/edges_is_of_age-checkpoint.csv to s3://cancer-knowledge-bucket/tcga-kg/.ipynb_checkpoints/edges_is_of_age-checkpoint.csv
upload: neptune_csv/.ipynb_checkpoints/vertices_subtype-checkpoint.csv to s3://cancer-knowledge-bucket/tcga-kg/.ipynb_checkpoints/vertices_subtype-checkpoint.csv
upload: neptune_csv/edges_has_subtype.csv to s3://cancer-knowledge-bucket/tcga-kg/edges_has_subtype.csv
upload: neptune_csv/.ipynb_checkpoints/edges_in_pathway-checkpoint.csv to s3://cancer-knowledge-bucket/tcga-kg/.ipynb_checkpoints/edges_in_pathway-checkpoint.csv
upload: neptune_csv/edges_is_of_age.csv to s3://cancer-knowledge-bucket/tcga-kg/edges_is_of_age.csv
upload: neptune_csv/.ipynb_checkpoints/vertices_age-checkpoint.csv to s3://cancer-knowledge-bucket/tcga-kg/.ipynb_checkpoints/vertices_age-checkpoin

# Commands for Neptune

In [255]:
# no need to run this
import aws_cdk.aws_neptune_alpha as neptune

### Uploading to Neptune

In [112]:
! curl -X POST \
    -H 'Content-Type: application/json' \
    https://db-neptune-1-instance-1.crc8kg8g0zaj.us-east-1.neptune.amazonaws.com:8182/loader -d '
    {
      "source" : "s3://cancer-knowledge-bucket/tcga-kg/",
      "format" : "csv",
      "iamRoleArn" : "arn:aws:iam::767398038972:role/NeptuneLoadFromS3",
      "region" : "us-east-1",
      "failOnError" : "FALSE",
      "parallelism" : "MEDIUM",
      "queueRequest" : "TRUE"
    }'

IndentationError: unexpected indent (1825805568.py, line 2)

### Query Neptune

In [ ]:
curl -X POST \
  -H 'Content-Type: application/json' \
  -d '{"gremlin": "g.V().limit(10)"}' \   
  https://db-neptune-1.cluster-crc8kg8g0zaj.us-east-1.neptune.amazonaws.com:8182/gremlin